<a href="https://colab.research.google.com/github/TaranSuratwala/STOCK-PREDICTION-PROJECT/blob/main/ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ta

In [ ]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing insta

SCREENER WITH MARKET CAPITALISATION

FOR ALL STOCKS

In [ ]:
import yfinance as yf
import pandas as pd
import ta
import warnings
warnings.filterwarnings("ignore")

# === Step 1: NSE Stock List ===
def get_nse_stocks():
    url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
    stocks_df = pd.read_csv(url)
    return stocks_df["SYMBOL"].tolist()

# === Step 2: Fetch Stock Data ===
def get_stock_data(symbol, period="2y", interval="1d"):
    try:
        ticker = yf.Ticker(symbol + ".NS")
        df = ticker.history(period=period, interval=interval)
        if df.empty:
            return None
        return df
    except:
        return None

# === Step 3: Compute Indicators (safe) ===
def compute_indicators(df, window=14):
    if df is None or len(df) < window + 1:
        return df

    # RSI
    df["RSI"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()

    # MACD
    macd = ta.trend.MACD(df["Close"])
    df["MACD"] = macd.macd()
    df["MACD_Signal"] = macd.macd_signal()

    # DMI & ADX
    if len(df) >= window + 1:
        try:
            adx = ta.trend.ADXIndicator(df["High"], df["Low"], df["Close"], window=window)
            df["+DMI"] = adx.adx_pos()
            df["-DMI"] = adx.adx_neg()
            df["ADX"] = adx.adx()
        except Exception:
            df["+DMI"], df["-DMI"], df["ADX"] = None, None, None

    # Volume MA
    if "Volume" in df:
        df["Vol_MA"] = df["Volume"].rolling(20).mean()

    return df

# === Step 4: Format Market Cap ===
def format_market_cap(market_cap):
    """Convert Market Cap into human readable units."""
    if market_cap is None:
        return None
    try:
        market_cap = float(market_cap)
        if market_cap >= 1e12:   # Lakh Crore
            return f"{market_cap/1e12:.2f} Lakh Cr"
        elif market_cap >= 1e7:  # Crore
            return f"{market_cap/1e7:.2f} Cr"
        elif market_cap >= 1e9:  # Billion (fallback for foreign listings)
            return f"{market_cap/1e9:.2f} Bn"
        else:
            return f"{market_cap:.0f}"
    except:
        return None

# === Step 5: Check Conditions ===
def check_conditions(symbol):
    df = get_stock_data(symbol)
    if df is None or len(df) < 50:
        return None

    df = compute_indicators(df)

    if "MACD" not in df or "RSI" not in df:
        return None

    # --- Daily ---
    latest = df.iloc[-1]
    prev = df.iloc[-2]

    cond1 = latest["MACD"] > 0 and latest["MACD"] > prev["MACD"]
    cond4 = latest["RSI"] > 60
    cond5 = "+DMI" in df and "-DMI" in df and latest["+DMI"] > latest["-DMI"]
    cond6 = "ADX" in df and latest["ADX"] > prev["ADX"]
    cond7 = "Vol_MA" in df and latest["Volume"] > latest["Vol_MA"]

    # --- Weekly ---
    df_weekly = df.resample("W").last()
    df_weekly = compute_indicators(df_weekly)
    cond2 = False
    if "MACD" in df_weekly and len(df_weekly) >= 3:
        latest_w, prev_w = df_weekly.iloc[-1], df_weekly.iloc[-2]
        cond2 = latest_w["MACD"] > 0 and latest_w["MACD"] > prev_w["MACD"]

    # --- Monthly ---
    df_monthly = df.resample("M").last()
    df_monthly = compute_indicators(df_monthly)
    cond3 = False
    if "MACD" in df_monthly and len(df_monthly) >= 3:
        latest_m, prev_m = df_monthly.iloc[-1], df_monthly.iloc[-2]
        cond3 = latest_m["MACD"] > 0 and latest_m["MACD"] > prev_m["MACD"]

    # === Market Capitalisation ===
    market_cap = None
    try:
        ticker = yf.Ticker(symbol + ".NS")
        market_cap = ticker.fast_info.get("market_cap", None)
        if market_cap is None:
            market_cap = ticker.info.get("marketCap", None)
    except:
        pass

    market_cap_fmt = format_market_cap(market_cap)

    conditions = {
        "Daily MACD Up": cond1,
        "Weekly MACD Up": cond2,
        "Monthly MACD Up": cond3,
        "RSI > 60": cond4,
        "+DMI > -DMI": cond5,
        "ADX Rising": cond6,
        "Vol > MA": cond7
    }

    satisfied = sum(conditions.values())

    return {
        "Symbol": symbol,
        "Market Cap": market_cap_fmt,    # formatted for display
        "Market Cap (Raw)": market_cap,  # numeric for sorting
        "Conditions Met": satisfied,
        **conditions
    }

# === Step 6: Main Screener ===
def screen_stocks():
    stock_list = get_nse_stocks()
    results = []

    for symbol in stock_list[:]:  # remove slicing to run full list
        result = check_conditions(symbol)
        if result:
            results.append(result)

    df = pd.DataFrame(results)

    if df.empty:
        return pd.DataFrame()

    if not (df["Conditions Met"] == 7).any():
        df = df[df["Conditions Met"] >= 4]

    # Sort by Conditions Met first, then Market Cap (numeric)
    df = df.sort_values(by=["Conditions Met", "Market Cap (Raw)"], ascending=[False, False])

    return df.reset_index(drop=True)

# === Run ===
df = screen_stocks()
df = df.drop(columns=["Market Cap (Raw)"])
print("\n📊 NSE Stock Screening Results:\n")
df.head(20)

KeyError: 'SYMBOL'

FOR NIFTY500 STOCKS

In [ ]:
import yfinance as yf
import pandas as pd
import ta
import warnings
warnings.filterwarnings("ignore")

# === Step 1: NIFTY 500 Stock List ===
def get_nse_stocks():
    """Fetch NIFTY 500 stock symbols and sector info from NSE"""
    url = "https://archives.nseindia.com/content/indices/ind_nifty500list.csv"
    stocks_df = pd.read_csv(url)
    return stocks_df[["Symbol", "Industry"]]

# === Step 2: Fetch Stock Data ===
def get_stock_data(symbol, period="2y", interval="1d"):
    try:
        ticker = yf.Ticker(symbol + ".NS")
        df = ticker.history(period=period, interval=interval)
        if df.empty:
            return None
        return df
    except:
        return None

# === Step 3: Compute Indicators (safe) ===
def compute_indicators(df, window=14):
    if df is None or len(df) < window + 1:
        return df

    # RSI
    df["RSI"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()

    # MACD
    macd = ta.trend.MACD(df["Close"])
    df["MACD"] = macd.macd()
    df["MACD_Signal"] = macd.macd_signal()

    # DMI & ADX
    if len(df) >= window + 1:
        try:
            adx = ta.trend.ADXIndicator(df["High"], df["Low"], df["Close"], window=window)
            df["+DMI"] = adx.adx_pos()
            df["-DMI"] = adx.adx_neg()
            df["ADX"] = adx.adx()
        except Exception:
            df["+DMI"], df["-DMI"], df["ADX"] = None, None, None

    # Volume MA
    if "Volume" in df:
        df["Vol_MA"] = df["Volume"].rolling(20).mean()

    return df

# === Step 4: Format Market Cap ===
def format_market_cap(market_cap):
    """Convert Market Cap into human readable units."""
    if market_cap is None:
        return None
    try:
        market_cap = float(market_cap)
        if market_cap >= 1e12:   # Lakh Crore
            return f"{market_cap/1e12:.2f} Lakh Cr"
        elif market_cap >= 1e7:  # Crore
            return f"{market_cap/1e7:.2f} Cr"
        elif market_cap >= 1e9:  # Billion (fallback for foreign listings)
            return f"{market_cap/1e9:.2f} Bn"
        else:
            return f"{market_cap:.0f}"
    except:
        return None

# === Step 5: Check Conditions ===
def check_conditions(symbol, sector):
    df = get_stock_data(symbol)
    if df is None or len(df) < 50:
        return None

    df = compute_indicators(df)

    if "MACD" not in df or "RSI" not in df:
        return None

    # --- Daily ---
    latest = df.iloc[-1]
    prev = df.iloc[-2]

    cond1 = latest["MACD"] > 0 and latest["MACD"] > prev["MACD"]
    cond4 = latest["RSI"] > 60
    cond5 = "+DMI" in df and "-DMI" in df and latest["+DMI"] > latest["-DMI"]
    cond6 = "ADX" in df and latest["ADX"] > prev["ADX"]
    cond7 = "Vol_MA" in df and latest["Volume"] > latest["Vol_MA"]

    # --- Weekly ---
    df_weekly = df.resample("W").last()
    df_weekly = compute_indicators(df_weekly)
    cond2 = False
    if "MACD" in df_weekly and len(df_weekly) >= 3:
        latest_w, prev_w = df_weekly.iloc[-1], df_weekly.iloc[-2]
        cond2 = latest_w["MACD"] > 0 and latest_w["MACD"] > prev_w["MACD"]

    # --- Monthly ---
    df_monthly = df.resample("M").last()
    df_monthly = compute_indicators(df_monthly)
    cond3 = False
    if "MACD" in df_monthly and len(df_monthly) >= 3:
        latest_m, prev_m = df_monthly.iloc[-1], df_monthly.iloc[-2]
        cond3 = latest_m["MACD"] > 0 and latest_m["MACD"] > prev_m["MACD"]

    # === Market Capitalisation ===
    market_cap = None
    try:
        ticker = yf.Ticker(symbol + ".NS")
        market_cap = ticker.fast_info.get("market_cap", None)
        if market_cap is None:
            market_cap = ticker.info.get("marketCap", None)
    except:
        pass

    market_cap_fmt = format_market_cap(market_cap)

    conditions = {
        "Daily MACD Up": cond1,
        "Weekly MACD Up": cond2,
        "Monthly MACD Up": cond3,
        "RSI > 60": cond4,
        "+DMI > -DMI": cond5,
        "ADX Rising": cond6,
        "Vol > MA": cond7
    }

    satisfied = sum(conditions.values())

    return {
        "Symbol": symbol,
        "Sector": sector,
        "Market Cap": market_cap_fmt,    # formatted for display
        "Market Cap (Raw)": market_cap,  # numeric for sorting
        "Conditions Met": satisfied,
        **conditions
    }

# === Step 6: Main Screener ===
def screen_stocks():
    stock_list = get_nse_stocks()
    results = []

    for _, row in stock_list.iterrows():
        symbol, sector = row["Symbol"], row["Industry"]
        result = check_conditions(symbol, sector)
        if result:
            results.append(result)

    df = pd.DataFrame(results)

    if df.empty:
        return pd.DataFrame()

    if not (df["Conditions Met"] == 7).any():
        df = df[df["Conditions Met"] >= 4]

    # Sort by Conditions Met first, then Market Cap (numeric)
    df = df.sort_values(by=["Conditions Met", "Market Cap (Raw)"], ascending=[False, False])

    return df.reset_index(drop=True)

# === Run ===
df = screen_stocks()
df = df.drop(columns=["Market Cap (Raw)"])
print("\n📊 NIFTY500 Stock Screening Results:\n")
df.head(20)

ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:$DUMMYDBRLT.NS: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")



📊 NIFTY500 Stock Screening Results:



,Symbol,Sector,Market Cap,Conditions Met,Daily MACD Up,Weekly MACD Up,Monthly MACD Up,RSI > 60,+DMI > -DMI,ADX Rising,Vol > MA
0,ADANIPOWER,Power,2.45 Lakh Cr,6,True,True,False,True,True,True,True
1,EICHERMOT,Automobile and Auto Components,1.88 Lakh Cr,6,True,True,False,True,True,True,True
2,HDFCAMC,Financial Services,1.24 Lakh Cr,6,True,True,False,True,True,True,True
3,CGPOWER,Capital Goods,1.21 Lakh Cr,6,True,True,False,True,True,True,True
4,CUMMINSIND,Capital Goods,1.12 Lakh Cr,6,True,True,False,True,True,True,True
5,WAAREEENER,Capital Goods,99911.28 Cr,6,True,True,False,True,True,True,True
6,INDIANB,Financial Services,93284.98 Cr,6,True,True,False,True,True,True,True
7,YESBANK,Financial Services,65189.35 Cr,6,True,True,False,True,True,True,True
8,MPHASIS,Information Technology,56473.22 Cr,6,True,True,False,True,True,True,True
9,BANKINDIA,Financial Services,53489.32 Cr,6,True,True,False,True,True,True,True


LSTM MODEL

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Fetch stock data
def fetch_stock_data(symbol):
    try:
        df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
        return df.dropna() if not df.empty else None
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return None

# Compute RSI, MACD, Bollinger Bands
def compute_indicators(df):
    close = df['Close']

    # RSI
    delta = close.diff()
    gain = delta.clip(lower=0).fillna(0)
    loss = -delta.clip(upper=0).fillna(0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD
    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26

    # Bollinger Bands
    sma_20 = close.rolling(window=20).mean()
    std_20 = close.rolling(window=20).std()
    df['BB_Middle'] = sma_20
    df['BB_Upper'] = sma_20 + 2 * std_20
    df['BB_Lower'] = sma_20 - 2 * std_20

    return df.dropna()

# Preprocessing
def preprocess(df, time_step=90):
    df = compute_indicators(df)
    features = ['Close', 'RSI', 'MACD', 'BB_Middle', 'BB_Upper', 'BB_Lower']
    df = df[features].dropna()

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(time_step, len(scaled)):
        X.append(scaled[i - time_step:i])
        y.append(scaled[i, 0])  # Close price

    return np.array(X), np.array(y), scaler, df.index[-len(y):]

# Inverse transform
def inverse_transform_close(scaled_close, scaler):
    dummy = np.zeros((scaled_close.shape[0], scaler.n_features_in_))
    dummy[:, 0] = scaled_close.ravel()
    return scaler.inverse_transform(dummy)[:, 0]

# Build model
def build_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(LSTM(64))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.002), loss='mean_squared_error')
    return model

# Predict future prices
def predict_future(model, last_seq, n_future, scaler):
    predictions = []
    seq = last_seq.copy()
    for _ in range(n_future):
        pred = model.predict(seq.reshape(1, *seq.shape), verbose=0)
        predictions.append(pred[0][0])
        next_step = np.roll(seq, -1, axis=0)
        next_step[-1, 0] = pred[0][0]  # Update only 'Close'
        next_step[-1, 1:] = seq[-1, 1:]  # Keep RSI, MACD, BBs static
        seq = next_step
    return inverse_transform_close(np.array(predictions).reshape(-1, 1), scaler)

# Plot actual vs predicted + future
def plot_graph(actual, predicted, future, date_range):
    test_dates = date_range[-len(predicted):]
    future_dates = pd.date_range(start=test_dates[-1] + timedelta(days=1), periods=len(future), freq='B')
    combined_dates = np.concatenate([test_dates, future_dates])
    combined_pred = np.concatenate([predicted.flatten(), future.flatten()])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_dates, y=actual, mode='lines', name='Actual', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=combined_dates, y=combined_pred, mode='lines', name='Predicted + Future', line=dict(color='red', dash='dash')))
    fig.update_layout(title='Stock Price Prediction (30 Days)', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')
    fig.show()

# Main function
def main():
    symbol = "CENTUM"  # Replace with your stock: "TATAPOWER", "INFY", etc.
    df = fetch_stock_data(symbol)
    if df is None:
        print("⚠️ No data available.")
        return

    time_step = 90 #taking last 90 rows as input
    X, y, scaler, date_range = preprocess(df, time_step)

    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    model = build_model((X_train.shape[1], X_train.shape[2]))

    early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, min_lr=1e-5)

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=32,
              callbacks=[early_stop, reduce_lr], verbose=1)

    pred_scaled = model.predict(X_test)
    actual = inverse_transform_close(y_test.reshape(-1, 1), scaler)
    predicted = inverse_transform_close(pred_scaled, scaler)

    future = predict_future(model, X[-1], n_future=30, scaler=scaler)
    plot_graph(actual, predicted, future, date_range)

    rmse = np.sqrt(mean_squared_error(actual, predicted))
    print(f"📉 RMSE: {rmse:.4f}")

if __name__ == "__main__":
    main()

Epoch 1/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 415ms/step - loss: 0.0437 - val_loss: 0.0165 - learning_rate: 0.0020
Epoch 2/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 349ms/step - loss: 0.0079 - val_loss: 0.0144 - learning_rate: 0.0020
Epoch 3/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 389ms/step - loss: 0.0057 - val_loss: 0.0188 - learning_rate: 0.0020
Epoch 4/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 381ms/step - loss: 0.0038 - val_loss: 0.0276 - learning_rate: 0.0020
Epoch 5/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 380ms/step - loss: 0.0048 - val_loss: 0.0281 - learning_rate: 0.0020
Epoch 6/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 391ms/step - loss: 0.0034 - val_loss: 0.0241 - learning_rate: 0.0020
Epoch 7/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 381ms/step - loss: 0.0026 - val_loss: 0.0258 - learning_rate: 6.0000e-04
Epoch 8/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 398ms/step - loss: 0.0021 - val_loss: 0.0268 - learning_rate: 6.0000e-04
Epoch 9/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 388ms/step - loss: 0.0029 - val_loss: 0.0362 - learni

📉 RMSE: 306.7341


TRANSFORMER + LSTM Model

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# ------------------------------
# Fetch stock data
# ------------------------------
def fetch_stock_data(symbol):
    try:
        df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
        return df.dropna() if not df.empty else None
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return None

# ------------------------------
# Compute indicators (RSI, MACD, Bollinger)
# ------------------------------
def compute_indicators(df):
    close = df['Close']

    # RSI
    delta = close.diff()
    gain = delta.clip(lower=0).fillna(0)
    loss = -delta.clip(upper=0).fillna(0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD
    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26

    # Bollinger Bands
    sma_20 = close.rolling(window=20).mean()
    std_20 = close.rolling(window=20).std()
    df['BB_Middle'] = sma_20
    df['BB_Upper'] = sma_20 + 2 * std_20
    df['BB_Lower'] = sma_20 - 2 * std_20

    return df.dropna()

# ------------------------------
# Preprocessing
# ------------------------------
def preprocess(df, time_step=90):
    df = compute_indicators(df)
    features = ['Close', 'RSI', 'MACD', 'BB_Middle', 'BB_Upper', 'BB_Lower']
    df = df[features].dropna()

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(time_step, len(scaled)):
        X.append(scaled[i - time_step:i])
        y.append(scaled[i, 0])  # Close price

    return np.array(X), np.array(y), scaler, df.index[-len(y):]

# ------------------------------
# Inverse transform
# ------------------------------
def inverse_transform_close(scaled_close, scaler):
    dummy = np.zeros((scaled_close.shape[0], scaler.n_features_in_))
    dummy[:, 0] = scaled_close.ravel()
    return scaler.inverse_transform(dummy)[:, 0]

# ------------------------------
# Positional Encoding (for Transformer)
# ------------------------------
def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    # apply sin to even indices, cos to odd
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    return tf.cast(angle_rads, dtype=tf.float32)

# ------------------------------
# Transformer Encoder Block
# ------------------------------
def transformer_encoder(inputs, num_heads=4, ff_dim=64, dropout=0.3):
    # Multi-Head Self Attention
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = Add()([inputs, attention])   # Residual
    attention = LayerNormalization()(attention)

    # Feed Forward
    ffn = Dense(ff_dim, activation="relu")(attention)
    ffn = Dense(inputs.shape[-1])(ffn)
    ffn = Dropout(dropout)(ffn)
    out = Add()([attention, ffn])  # Residual
    out = LayerNormalization()(out)

    return out

# ------------------------------
# Build Hybrid Model (LSTM + Transformer)
# ------------------------------
def build_hybrid_model(input_shape):
    inputs = Input(shape=input_shape)

    # Add positional encoding
    seq_len = input_shape[0]
    d_model = input_shape[1]
    pos_encoding = positional_encoding(seq_len, d_model)
    x = inputs + pos_encoding

    # BiLSTM branch
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    # Transformer block
    x = transformer_encoder(x, num_heads=4, ff_dim=128, dropout=0.3)

    # Final LSTM to compress
    x = LSTM(64)(x)
    x = Dropout(0.3)(x)

    # Dense layers
    x = Dense(64, activation="relu")(x)
    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.002), loss="mean_squared_error")

    return model

# ------------------------------
# Predict future prices
# ------------------------------
def predict_future(model, last_seq, n_future, scaler):
    predictions = []
    seq = last_seq.copy()
    for _ in range(n_future):
        pred = model.predict(seq.reshape(1, *seq.shape), verbose=0)
        predictions.append(pred[0][0])
        next_step = np.roll(seq, -1, axis=0)
        next_step[-1, 0] = pred[0][0]  # update only Close
        next_step[-1, 1:] = seq[-1, 1:]  # keep RSI, MACD, BBs static
        seq = next_step
    return inverse_transform_close(np.array(predictions).reshape(-1, 1), scaler)

# ------------------------------
# Plot actual vs predicted + future
# ------------------------------
def plot_graph(actual, predicted, future, date_range):
    test_dates = date_range[-len(predicted):]
    future_dates = pd.date_range(start=test_dates[-1] + timedelta(days=1), periods=len(future), freq='B')
    combined_dates = np.concatenate([test_dates, future_dates])
    combined_pred = np.concatenate([predicted.flatten(), future.flatten()])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_dates, y=actual, mode='lines', name='Actual', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=combined_dates, y=combined_pred, mode='lines', name='Predicted + Future', line=dict(color='red', dash='dash')))
    fig.update_layout(title='Stock Price Prediction (30 Days)', xaxis_title='Date', yaxis_title='Price', template='plotly_dark')
    fig.show()

# ------------------------------
# Main function
# ------------------------------
def main():
    symbol = "TATAPOWER"  # change stock symbol here
    df = fetch_stock_data(symbol)
    if df is None:
        print("⚠️ No data available.")
        return

    time_step = 90
    X, y, scaler, date_range = preprocess(df, time_step)

    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    model = build_hybrid_model((X_train.shape[1], X_train.shape[2]))

    early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, min_lr=1e-5)

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=32,
              callbacks=[early_stop, reduce_lr], verbose=1)

    pred_scaled = model.predict(X_test)
    actual = inverse_transform_close(y_test.reshape(-1, 1), scaler)
    predicted = inverse_transform_close(pred_scaled, scaler)

    future = predict_future(model, X[-1], n_future=30, scaler=scaler)
    plot_graph(actual, predicted, future, date_range)

    rmse = np.sqrt(mean_squared_error(actual, predicted))
    print(f"📉 RMSE: {rmse:.4f}")

if __name__ == "__main__":
    main()

Epoch 1/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 855ms/step - loss: 0.1377 - val_loss: 0.0832 - learning_rate: 0.0020
Epoch 2/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 25s 860ms/step - loss: 0.0147 - val_loss: 0.0770 - learning_rate: 0.0020
Epoch 3/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 814ms/step - loss: 0.0118 - val_loss: 0.0567 - learning_rate: 0.0020
Epoch 4/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 790ms/step - loss: 0.0121 - val_loss: 0.0043 - learning_rate: 0.0020
Epoch 5/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 25s 849ms/step - loss: 0.0102 - val_loss: 0.0039 - learning_rate: 0.0020
Epoch 6/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 820ms/step - loss: 0.0074 - val_loss: 0.0033 - learning_rate: 0.0020
Epoch 7/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 823ms/step - loss: 0.0049 - val_loss: 0.0024 - learning_rate: 0.0020
Epoch 8/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 786ms/step - loss: 0.0061 - val_loss: 0.0052 - learning_rate: 0.0020
Epoch 9/150
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 844ms/step - loss: 0.0056 - val_loss: 0.0163 - learning_rate:

📉 RMSE: 20.9806


Trials

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# --------------------
# Global Constants
# --------------------
CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# List of features for the model
features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200"]


# --------------------
# Fetch & cache data
# --------------------
def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    # Accept 'TATAPOWER' or 'TATAPOWER.NS'
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol
    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")

    if os.path.exists(cache_path) and not force_download:
        df = pd.read_csv(cache_path, index_col=0, parse_dates=True)
        print(f"Loaded cached data: {cache_path}")
        return df
    df = yf.download(yf_symbol, period=period, interval=interval, progress=False)
    if df is None or df.empty:
        raise ValueError(f"No data found for {yf_symbol}")
    df.to_csv(cache_path)
    print(f"Downloaded and cached: {cache_path}")
    return df


# --------------------
# Indicators (ensure 1D inputs)
# --------------------
def compute_indicators(df):
    df = df.copy()
    # Ensure all required columns are numeric to prevent TypeErrors
    for col in ["Close", "High", "Low", "Volume", "Open"]:
        if col in df.columns:
            ser = df[col]
            # If selecting the column returned a DataFrame (duplicate column names), pick the first column
            if isinstance(ser, pd.DataFrame):
                ser = ser.iloc[:, 0]
            # If it's not already a Series (e.g., numpy array or list), coerce into a Series with original index
            if not isinstance(ser, pd.Series):
                try:
                    ser = pd.Series(ser, index=df.index)
                except Exception:
                    ser = pd.Series(np.asarray(ser).ravel())
            df[col] = pd.to_numeric(ser, errors='coerce')
    df = df.dropna(subset=["Close", "High", "Low", "Volume"])
    close = df["Close"]
    high = df["High"]
    low = df["Low"]
    vol = df["Volume"]
    # RSI, MACD, Bollinger mid, ADX/DIs, OBV, MA/EMA
    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()

    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()

    return df.dropna()


# --------------------
# Build sequences, split, scale (fit scalers on TRAIN only)
# --------------------
def make_sequences_and_scales(df, window=90, train_ratio=0.8):
    # compute indicators and keep a deterministic feature order
    df_ind = compute_indicators(df)

    df_feat = df_ind[features].copy()
    if len(df_feat) < window + 10:
        raise ValueError("Not enough data after indicators to form sequences. Reduce window or get more data.")

    X_raw = []
    y_raw = []
    last_close_raw = []
    dates = []
    open_prices = []
    high_prices = []
    low_prices = []
    close_prices = []

    for i in range(window, len(df_ind)):
        seq = df_feat.iloc[i - window:i].values  # shape (window, n_features)
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])    # raw close price at target time
        last_close_raw.append(df_feat["Close"].iloc[i - 1])  # persistence baseline
        dates.append(df_ind.index[i])
        open_prices.append(df_ind["Open"].iloc[i])
        high_prices.append(df_ind["High"].iloc[i])
        low_prices.append(df_ind["Low"].iloc[i])
        close_prices.append(df_ind["Close"].iloc[i])

    X_raw = np.array(X_raw, dtype=np.float32)      # (samples, window, features)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)
    # keep dates as pandas Timestamps list (not auto-converted to str)
    dates = np.array(dates, dtype='datetime64[ns]')
    open_prices = np.array(open_prices, dtype=np.float32)
    high_prices = np.array(high_prices, dtype=np.float32)
    low_prices = np.array(low_prices, dtype=np.float32)
    close_prices = np.array(close_prices, dtype=np.float32)


    # train/test split by time
    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = dates[split:]
    test_open_prices = open_prices[split:]
    test_high_prices = high_prices[split:]
    test_low_prices = low_prices[split:]
    test_close_prices = close_prices[split:]

    # Fit feature scaler on TRAIN (flatten window dimension)
    n_features = X_train_raw.shape[2]
    X_train_flat = X_train_raw.reshape(-1, n_features)
    X_test_flat = X_test_raw.reshape(-1, n_features)

    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_flat)

    X_train_scaled = feature_scaler.transform(X_train_flat).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_flat).reshape(X_test_raw.shape)

    # Fit target scaler on TRAIN targets (minmax gives bounded outputs)
    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)  # shape (n_samples,1)

    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            test_open_prices, test_high_prices, test_low_prices, test_close_prices,
            feature_scaler, target_scaler)


# --------------------
# Model (Functional API): Conv1D -> BiLSTM -> Attention -> Pooling -> Dense
# --------------------
def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, padding="causal", activation="relu")(inp)
    x = Dropout(0.1)(x)

    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    # self-attention over sequence
    att = MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)

    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    out = Dense(1)(x)  # predict scaled target (MinMax)

    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model


# --------------------
# Train, predict, evaluate, plot
# --------------------
def run(symbol="TATAPOWER", window=90, epochs=50, batch_size=32, force_download=False, n_forecast_days=30):
    # fetch
    print("Fetching data...")
    df = fetch_data(symbol, force_download=force_download)
    print("Raw rows:", len(df))

    # sequences and scalers
    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     test_open_prices, test_high_prices, test_low_prices, test_close_prices,
     feature_scaler, target_scaler) = make_sequences_and_scales(df, window=window)

    print("Shapes -> X_train:", X_train.shape, "X_test:", X_test.shape)

    # build & train
    model = build_model((X_train.shape[1], X_train.shape[2]), lr=1e-3)
    model.summary()

    es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)
    history = model.fit(X_train, y_train_scaled,
                        validation_data=(X_test, y_test_scaled),
                        epochs=epochs, batch_size=batch_size, callbacks=[es],
                        verbose=2)

    # predict on test set (scaled -> inverse)
    pred_scaled = model.predict(X_test).reshape(-1, 1)
    pred_price = target_scaler.inverse_transform(pred_scaled).reshape(-1)  # real price units
    y_test_price = y_test_raw  # already raw

    # baselines
    persist_pred = last_close_test  # persistence baseline (last observed close)
    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, persist_pred))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1] if len(y_test_price) > 1 else np.nan

    print("\nMetrics (price units):")
    print(f" Model RMSE       : {rmse_model:.4f}")
    print(f" Persistence RMSE: {rmse_persist:.4f}")
    print(f" Corr (actual vs pred): {corr:.4f}")

    # Forecast for the next N days
    forecast_prices = []
    df_for_forecast = compute_indicators(df)
    last_window_data = df_for_forecast.iloc[-window:][features].copy()

    # ensure last_window_data index is datetime
    last_window_data.index = pd.to_datetime(last_window_data.index)

    for _ in range(n_forecast_days):
        scaled_last_window = feature_scaler.transform(last_window_data.values).reshape(1, window, -1)
        pred_scaled_next_day = model.predict(scaled_last_window, verbose=0)
        pred_price_next_day = target_scaler.inverse_transform(pred_scaled_next_day).flatten()[0]
        forecast_prices.append(float(pred_price_next_day))

        # build new row for the rolling window: use predicted close and keep other features as previous (simple approach)
        new_row = last_window_data.iloc[-1].copy()
        new_row['Close'] = pred_price_next_day
        # shift index by 1 day
        new_index = last_window_data.index[-1] + pd.Timedelta(days=1)
        new_row_df = pd.DataFrame([new_row], index=[new_index])
        last_window_data = pd.concat([last_window_data.iloc[1:], new_row_df], axis=0)

    # Ensure test_dates are datetimes
    test_dates = pd.to_datetime(test_dates)

    # Forecast dates as DatetimeIndex starting the day after last test date
    start_forecast_date = pd.to_datetime(test_dates[-1]) + pd.Timedelta(days=1)
    forecast_dates = pd.date_range(start=start_forecast_date, periods=n_forecast_days, freq='D')

    # Save model
    safe_symbol = symbol.replace(".", "_")
    model_path = os.path.join(MODELS_DIR, f"{safe_symbol}_improved.keras")
    model.save(model_path)
    print("Saved model to:", model_path)

    # Plotting: correctly combine test and forecast data for a continuous line
    fig = go.Figure()

    # Add candlestick chart for the test period
    fig.add_trace(go.Candlestick(x=test_dates,
                                 open=test_open_prices,
                                 high=test_high_prices,
                                 low=test_low_prices,
                                 close=test_close_prices,
                                 name="Actual Price (Candlestick)"))

    # Plot predicted values for the test set
    fig.add_trace(go.Scatter(x=test_dates, y=pred_price, mode="lines", name="Predicted Price",
                             line=dict(width=2, dash="dash")))

    # Build forecast x as a list of datetimes for compatibility
    forecast_start_date = pd.to_datetime(test_dates[-1])
    forecast_x = [forecast_start_date] + list(forecast_dates)
    # Combine last predicted value of the test set with forecast values
    forecast_y = [float(pred_price[-1])] + forecast_prices

    fig.add_trace(go.Scatter(x=forecast_x, y=forecast_y, mode="lines", name="Forecast",
                             line=dict(width=2, dash="solid")))

    # Add a vertical line to mark the end of the test set and start of the forecast
    fig.add_shape(type="line",
                  x0=forecast_start_date, y0=0, x1=forecast_start_date, y1=1,
                  yref='paper', line=dict(width=2, dash="dot"))

    # Add baselines
    fig.add_trace(go.Scatter(x=test_dates, y=persist_pred, mode="lines", name="Persistence (last close)",
                             line=dict(width=1, dash="dot")))

    fig.update_layout(title=f"{symbol} — Actual vs Predicted & Forecast",
                      xaxis_title="Date", yaxis_title="Price",
                      template="plotly_dark", hovermode="x unified",
                      legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

    plot_path = os.path.join(REPORTS_DIR, f"{safe_symbol}_prediction_plot.html")
    fig.write_html(plot_path)
    print("Saved interactive plot to:", plot_path)

    # show (works in notebooks or will open a browser in GUI env)
    try:
        fig.show()
    except Exception:
        print("Could not display plot inline (headless environment). Open HTML:", plot_path)

    # return useful outputs
    return {
        "model_path": model_path,
        "plot_path": plot_path,
        "rmse_model": float(rmse_model),
        "rmse_persist": float(rmse_persist),
        "corr": float(corr),
        "pred_price": pred_price,
        "y_test_price": y_test_price,
        "test_dates": test_dates,
        "forecast_prices": forecast_prices,
        "forecast_dates": forecast_dates
    }


if __name__ == "__main__":
    stock_name = input("Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): ").strip()
    if not stock_name:
        stock_name = "TATAPOWER"  # default fallback
    res = run(symbol=stock_name, window=90, epochs=50, batch_size=32, force_download=False, n_forecast_days=30)
    print("Done. Summary:", {k: res[k] for k in ("rmse_model", "rmse_persist", "corr", "model_path", "plot_path")})

Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): RELIANCE
Fetching data...
Loaded cached data: cache_data/RELIANCE_NS_5y_1d.csv
Raw rows: 1241
Shapes -> X_train: (892, 90, 15) X_test: (224, 90, 15)


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 90, 15)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 90, 64)    │      2,944 │ input_layer_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_48          │ (None, 90, 64)    │          0 │ conv1d_16[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_16    │ (None, 90, 128)   │     66,048 │ dropout_48[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 90, 128)   │     33,088 │ bidirectional_16… │
│ (MultiHeadAttentio… │                   │            │ bidirectional_16… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 90, 128)   │          0 │ bidirectional_16… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 90, 128)   │        256 │ add_16[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 64)        │      8,256 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_50          │ (None, 64)        │          0 │ dense_32[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 1)         │         65 │ dropout_50[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 110,657 (432.25 KB)

 Trainable params: 110,657 (432.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
28/28 - 11s - 380ms/step - loss: 0.2767 - val_loss: 0.0676
Epoch 2/50
28/28 - 6s - 207ms/step - loss: 0.0331 - val_loss: 0.0373
Epoch 3/50
28/28 - 3s - 123ms/step - loss: 0.0161 - val_loss: 0.0415
Epoch 4/50
28/28 - 6s - 203ms/step - loss: 0.0108 - val_loss: 0.0368
Epoch 5/50
28/28 - 5s - 166ms/step - loss: 0.0084 - val_loss: 0.0329
Epoch 6/50
28/28 - 5s - 184ms/step - loss: 0.0073 - val_loss: 0.0498
Epoch 7/50
28/28 - 6s - 220ms/step - loss: 0.0073 - val_loss: 0.0818
Epoch 8/50
28/28 - 4s - 141ms/step - loss: 0.0065 - val_loss: 0.0541
Epoch 9/50
28/28 - 4s - 127ms/step - loss: 0.0067 - val_loss: 0.0502
Epoch 10/50
28/28 - 5s - 172ms/step - loss: 0.0063 - val_loss: 0.0519
Epoch 11/50
28/28 - 4s - 127ms/step - loss: 0.0069 - val_loss: 0.0540
Epoch 12/50
28/28 - 5s - 194ms/step - loss: 0.0058 - val_loss: 0.0385
Epoch 13/50
28/28 - 5s - 162ms/step - loss: 0.0064 - val_loss: 0.0491
Epoch 14/50
28/28 - 4s - 153ms/step - loss: 0.0054 - val_loss: 0.0501
Epoch 15/50
28/28 - 6s - 226

Done. Summary: {'rmse_model': 131.8720304760452, 'rmse_persist': 17.313009426733306, 'corr': 0.5514455940698304, 'model_path': 'models/RELIANCE_improved.keras', 'plot_path': 'reports/RELIANCE_prediction_plot.html'}


In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# --------------------
# Global Constants
# --------------------
CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Features used in model
features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200"]


# --------------------
# Fetch & cache data
# --------------------
def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    """Fetch and cache OHLCV data from Yahoo Finance"""
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol

    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")

    if os.path.exists(cache_path) and not force_download:
        df = pd.read_csv(cache_path, index_col=0, parse_dates=True)
        print(f"Loaded cached data: {cache_path}")
    else:
        df = yf.download(yf_symbol, period=period, interval=interval, progress=False)
        if df is None or df.empty:
            raise ValueError(f"No data found for {yf_symbol}")
        df.to_csv(cache_path)
        print(f"Downloaded and cached: {cache_path}")

    # Ensure standard OHLCV columns
    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Missing column '{col}' in fetched data")

    df = df[required_cols]  # keep only OHLCV
    df = df.dropna()
    return df


# --------------------
# Indicators
# --------------------
def compute_indicators(df):
    df = df.copy()

    # Ensure numeric columns
    for col in ["Close", "High", "Low", "Volume", "Open"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["Close", "High", "Low", "Volume"])
    close, high, low, vol = df["Close"], df["High"], df["Low"], df["Volume"]

    # RSI
    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # MACD
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()

    # Bollinger Bands
    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()

    # ADX/DIs
    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()

    # OBV
    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()

    # MAs
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()

    return df.dropna()


# --------------------
# Sequences and scaling
# --------------------
def make_sequences_and_scales(df, window=90, train_ratio=0.8):
    df_ind = compute_indicators(df)
    df_feat = df_ind[features].copy()

    if len(df_feat) < window + 10:
        raise ValueError("Not enough data for sequences. Try reducing window or increasing history.")

    X_raw, y_raw, last_close_raw, dates = [], [], [], []
    open_prices, high_prices, low_prices, close_prices = [], [], [], []

    for i in range(window, len(df_ind)):
        seq = df_feat.iloc[i - window:i].values
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])
        last_close_raw.append(df_feat["Close"].iloc[i - 1])
        dates.append(df_ind.index[i])
        open_prices.append(df_ind["Open"].iloc[i])
        high_prices.append(df_ind["High"].iloc[i])
        low_prices.append(df_ind["Low"].iloc[i])
        close_prices.append(df_ind["Close"].iloc[i])

    X_raw = np.array(X_raw, dtype=np.float32)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)

    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = np.array(dates[split:], dtype="datetime64[ns]")
    test_open_prices = np.array(open_prices[split:], dtype=np.float32)
    test_high_prices = np.array(high_prices[split:], dtype=np.float32)
    test_low_prices = np.array(low_prices[split:], dtype=np.float32)
    test_close_prices = np.array(close_prices[split:], dtype=np.float32)

    n_features = X_train_raw.shape[2]
    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_raw.reshape(-1, n_features))

    X_train_scaled = feature_scaler.transform(X_train_raw.reshape(-1, n_features)).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_raw.reshape(-1, n_features)).reshape(X_test_raw.shape)

    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)

    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            test_open_prices, test_high_prices, test_low_prices, test_close_prices,
            feature_scaler, target_scaler)


# --------------------
# Model
# --------------------
def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)
    x = Conv1D(128, 5, padding="causal", activation="relu")(inp)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    att = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1)(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model


# --------------------
# Training & forecasting
# --------------------
def run(symbol="TATAPOWER", window=90, epochs=80, batch_size=32,
        force_download=False, n_forecast_days=30, ensemble_n=1):

    df = fetch_data(symbol, force_download=force_download)
    print("Data rows:", len(df))

    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     test_open, test_high, test_low, test_close,
     feature_scaler, target_scaler) = make_sequences_and_scales(df, window)

    models = []
    for i in range(ensemble_n):
        model = build_model((X_train.shape[1], X_train.shape[2]))
        es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
        model.fit(X_train, y_train_scaled, validation_data=(X_test, y_test_scaled),
                  epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=2)
        models.append(model)

    preds = np.mean([m.predict(X_test).reshape(-1, 1) for m in models], axis=0)
    pred_price = target_scaler.inverse_transform(preds).flatten()
    y_test_price = y_test_raw

    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, last_close_test))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1]

    print(f"Model RMSE: {rmse_model:.4f}, Persistence RMSE: {rmse_persist:.4f}, Corr: {corr:.4f}")

    return {"rmse_model": float(rmse_model), "rmse_persist": float(rmse_persist), "corr": float(corr)}


if __name__ == "__main__":
    stock_name = input("Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): ").strip() or "TATAPOWER"
    res = run(symbol=stock_name, window=90, epochs=80, batch_size=32,
              force_download=False, n_forecast_days=30, ensemble_n=1)
    print("Summary:", res)

Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): TATAPOWER
Loaded cached data: cache_data/TATAPOWER_NS_5y_1d.csv
Data rows: 1240
Epoch 1/80
28/28 - 21s - 764ms/step - loss: 0.8545 - val_loss: 0.0070
Epoch 2/80
28/28 - 20s - 701ms/step - loss: 0.0232 - val_loss: 0.0087
Epoch 3/80
28/28 - 14s - 508ms/step - loss: 0.0128 - val_loss: 0.0114
Epoch 4/80
28/28 - 14s - 517ms/step - loss: 0.0114 - val_loss: 0.0048
Epoch 5/80
28/28 - 21s - 737ms/step - loss: 0.0112 - val_loss: 0.0038
Epoch 6/80
28/28 - 16s - 567ms/step - loss: 0.0102 - val_loss: 0.0039
Epoch 7/80
28/28 - 20s - 711ms/step - loss: 0.0091 - val_loss: 0.0051
Epoch 8/80
28/28 - 20s - 705ms/step - loss: 0.0090 - val_loss: 0.0049
Epoch 9/80
28/28 - 15s - 536ms/step - loss: 0.0092 - val_loss: 0.0280
Epoch 10/80
28/28 - 20s - 715ms/step - loss: 0.0082 - val_loss: 0.0035
Epoch 11/80
28/28 - 22s - 788ms/step - loss: 0.0078 - val_loss: 0.0036
Epoch 12/80
28/28 - 20s - 700ms/step - loss: 0.0081 - val_loss: 0.0025
Epoch 13/80
28/28 - 20s

In [ ]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing insta

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import pandas_ta as pta  # For candlestick pattern detection
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# --------------------
# Global Constants
# --------------------
CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Features used in model
features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200"]

# --------------------
# Fetch & cache data
# --------------------
def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol

    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")

    if os.path.exists(cache_path) and not force_download:
        df = pd.read_csv(cache_path, index_col=0)
        df.index = pd.to_datetime(df.index, errors="coerce")
        df = df[~df.index.isna()]
        print(f"Loaded cached data: {cache_path}")
    else:
        df = yf.download(yf_symbol, period=period, interval=interval, progress=False)
        if df is None or df.empty:
            raise ValueError(f"No data found for {yf_symbol}")
        df.to_csv(cache_path)
        print(f"Downloaded and cached: {cache_path}")

    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Missing column '{col}' in fetched data")

    df = df[required_cols].dropna()
    return df

# --------------------
# Indicators
# --------------------
def compute_indicators(df):
    df = df.copy()
    for col in ["Close", "High", "Low", "Volume", "Open"]:
        val = df[col]
        if isinstance(val, pd.DataFrame):
            if col in val.columns:
                val = val[col]
            else:
                val = val.iloc[:, 0]
        df[col] = pd.to_numeric(val, errors="coerce")

    df = df.dropna(subset=["Close", "High", "Low", "Volume"])
    close, high, low, vol = df["Close"], df["High"], df["Low"], df["Volume"]

    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()

    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()

    return df.dropna()

# --------------------
# Sequences and scaling (All past data considered)
# --------------------
def make_sequences_and_scales(df, window=90, train_ratio=0.8):
    df_ind = compute_indicators(df)
    df_feat = df_ind[features].copy()
    n_features = len(features)

    X_raw, y_raw, last_close_raw, dates = [], [], [], []

    for i in range(1, len(df_feat)):
        seq = df_feat.iloc[:i].values  # all past data up to current day
        if len(seq) < window:
            pad_len = window - len(seq)
            seq = np.vstack([np.zeros((pad_len, n_features)), seq])
        else:
            seq = seq[-window:]  # take last `window` rows
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])
        last_close_raw.append(df_feat["Close"].iloc[i - 1])
        dates.append(df_ind.index[i])

    X_raw = np.array(X_raw, dtype=np.float32)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)

    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = np.array(dates[split:], dtype="datetime64[ns]")

    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_raw.reshape(-1, n_features))
    X_train_scaled = feature_scaler.transform(X_train_raw.reshape(-1, n_features)).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_raw.reshape(-1, n_features)).reshape(X_test_raw.shape)

    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)
    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            feature_scaler, target_scaler, df_ind)

# --------------------
# Model
# --------------------
def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)
    x = Conv1D(128, 5, padding="causal", activation="relu")(inp)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    att = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1)(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model

# --------------------
# Training & forecasting
# --------------------
def run(symbol="TATAPOWER", window=90, epochs=80, batch_size=32,
        force_download=False, n_forecast_days=30, ensemble_n=1):

    df = fetch_data(symbol, force_download=force_download)
    print("Data rows:", len(df))

    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     feature_scaler, target_scaler, df_ind) = make_sequences_and_scales(df, window)

    models = []
    for i in range(ensemble_n):
        model = build_model((X_train.shape[1], X_train.shape[2]))
        es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
        model.fit(X_train, y_train_scaled, validation_data=(X_test, y_test_scaled),
                  epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=2)
        models.append(model)

    preds = np.mean([m.predict(X_test).reshape(-1, 1) for m in models], axis=0)
    pred_price = target_scaler.inverse_transform(preds).flatten()
    y_test_price = y_test_raw

    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, last_close_test))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1]

    print(f"Model RMSE: {rmse_model:.4f}, Persistence RMSE: {rmse_persist:.4f}, Corr: {corr:.4f}")

    # Next 30-day Forecast
    last_seq = df_ind[features].iloc[-window:].values
    last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)

    future_preds = []
    for _ in range(n_forecast_days):
        step_preds = np.mean([m.predict(last_seq_scaled) for m in models], axis=0)
        step_price = target_scaler.inverse_transform(step_preds).flatten()[0]
        future_preds.append(step_price)

        new_row = last_seq[-1].copy()
        new_row[features.index("Close")] = step_price
        last_seq = np.vstack([last_seq[1:], new_row])
        last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)

    future_dates = pd.date_range(df.index[-1], periods=n_forecast_days+1, freq="B")[1:]

    # --------------------
    # Detect all candlestick patterns
    # --------------------
    pattern_names = [attr for attr in dir(pta.cdl) if attr.startswith("cdl_")]
    pattern_signals = {}
    for pattern in pattern_names:
        func = getattr(pta.cdl, pattern)
        df_ind[pattern.upper()] = func(df_ind["Open"], df_ind["High"], df_ind["Low"], df_ind["Close"])
        pattern_signals[pattern.upper()] = df_ind[df_ind[pattern.upper()] != 0]

    # Candlestick Plot (last 2 years)
    df_plot = df[df.index >= (df.index[-1] - pd.Timedelta(days=730))]
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot.index,
        open=df_plot["Open"],
        high=df_plot["High"],
        low=df_plot["Low"],
        close=df_plot["Close"],
        name="Candlestick"
    )])

    # Overlay all bullish/bearish markers
    for pattern_name, pattern_df in pattern_signals.items():
        bullish = pattern_df[pattern_df[pattern_name] > 0]
        bearish = pattern_df[pattern_df[pattern_name] < 0]

        fig.add_trace(go.Scatter(
            x=bullish.index,
            y=bullish["Close"],
            mode="markers",
            marker=dict(color="green", size=10, symbol="triangle-up"),
            name=f"Bullish {pattern_name}",
            text=[pattern_name]*len(bullish),
            hoverinfo="text+y"
        ))
        fig.add_trace(go.Scatter(
            x=bearish.index,
            y=bearish["Close"],
            mode="markers",
            marker=dict(color="red", size=10, symbol="triangle-down"),
            name=f"Bearish {pattern_name}",
            text=[pattern_name]*len(bearish),
            hoverinfo="text+y"
        ))

    # Add test predictions & future forecast
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=pred_price,
        mode="lines",
        name="Predicted (Test)",
        line=dict(color="blue", dash="dot")
    ))
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=y_test_price,
        mode="lines",
        name="Actual (Test)",
        line=dict(color="orange")
    ))
    fig.add_trace(go.Scatter(
        x=future_dates,
        y=future_preds,
        mode="lines",
        name="Forecast (Next 30d)",
        line=dict(color="green", dash="dash")
    ))

    fig.update_layout(
        title=f"{symbol} Candlestick (Last 2 Years) with Predictions, 30-Day Forecast & All Patterns",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.show()

    return {"rmse_model": float(rmse_model), "rmse_persist": float(rmse_persist), "corr": float(corr)}

if __name__ == "__main__":
    stock_name = input("Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): ").strip() or "TATAPOWER"
    res = run(symbol=stock_name, window=90, epochs=80, batch_size=32,
              force_download=False, n_forecast_days=30, ensemble_n=1)
    print("Summary:", res)

Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): TATAPOWER
Loaded cached data: cache_data/TATAPOWER_NS_5y_1d.csv
Data rows: 1239
Epoch 1/80
31/31 - 23s - 732ms/step - loss: 0.3301 - val_loss: 0.0102
Epoch 2/80
31/31 - 19s - 615ms/step - loss: 0.0185 - val_loss: 0.0178
Epoch 3/80
31/31 - 16s - 501ms/step - loss: 0.0154 - val_loss: 0.0051
Epoch 4/80
31/31 - 16s - 501ms/step - loss: 0.0124 - val_loss: 0.0034
Epoch 5/80
31/31 - 17s - 546ms/step - loss: 0.0101 - val_loss: 0.0040
Epoch 6/80
31/31 - 19s - 625ms/step - loss: 0.0102 - val_loss: 0.0070
Epoch 7/80
31/31 - 17s - 553ms/step - loss: 0.0091 - val_loss: 0.0024
Epoch 8/80
31/31 - 17s - 533ms/step - loss: 0.0083 - val_loss: 0.0042
Epoch 9/80
31/31 - 15s - 497ms/step - loss: 0.0078 - val_loss: 0.0024
Epoch 10/80
31/31 - 15s - 499ms/step - loss: 0.0088 - val_loss: 0.0092
Epoch 11/80
31/31 - 16s - 507ms/step - loss: 0.0086 - val_loss: 0.0022
Epoch 12/80
31/31 - 22s - 703ms/step - loss: 0.0088 - val_loss: 0.0099
Epoch 13/80
31/31 - 16s

Summary: {'rmse_model': 15.404509718836701, 'rmse_persist': 6.409094797572242, 'corr': 0.8286994624368046}


In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import pandas_ta as pta  # For candlestick pattern detection
import plotly.graph_objects as go
from tqdm import tqdm  # <-- Added for progress bars

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# --------------------
# Global Constants
# --------------------
CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200",
            "ATR", "stoch_k", "stoch_d", "CCI"]

# --------------------
# Fetch & cache data
# --------------------
def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol

    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")

    def download_and_cache():
        df_new = yf.download(yf_symbol, period=period, interval=interval, progress=False)
        if df_new is None or df_new.empty:
            raise ValueError(f"No data found for {yf_symbol}")
        df_new.to_csv(cache_path)
        print(f"Downloaded and cached: {cache_path}")
        return df_new

    def validate_cache(df_cache):
        required_cols = ["Open", "High", "Low", "Close", "Volume"]
        if not all(col in df_cache.columns for col in required_cols):
            return False
        if df_cache[required_cols].isnull().all().any():
            return False
        return True

    if os.path.exists(cache_path) and not force_download:
        try:
            df = pd.read_csv(cache_path, index_col=0)
            df.index = pd.to_datetime(df.index, errors="coerce")
            df = df[~df.index.isna()]
            if not validate_cache(df):
                print(f"Cache corrupted or incomplete. Deleting and re-downloading: {cache_path}")
                os.remove(cache_path)
                df = download_and_cache()
            else:
                print(f"Loaded cached data: {cache_path}")
        except Exception:
            print(f"Failed to read cache. Deleting and re-downloading: {cache_path}")
            if os.path.exists(cache_path):
                os.remove(cache_path)
            df = download_and_cache()
    else:
        df = download_and_cache()

    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    df = df.dropna(subset=[col for col in required_cols if col in df.columns])
    return df

# --------------------
# Indicators
# --------------------
def compute_indicators(df):
    df = df.copy()
    cols_to_coerce = ["Close", "High", "Low", "Volume", "Open"]
    for col in cols_to_coerce:
        if col in df.columns:
            val = df[col]
            if isinstance(val, pd.DataFrame):
                if col in val.columns:
                    val = val[col]
                else:
                    val = val.iloc[:, 0]
            df[col] = pd.to_numeric(val, errors="coerce")

    existing_cols = [c for c in cols_to_coerce if c in df.columns]
    if existing_cols:
        df = df.dropna(subset=existing_cols)

    close, high, low, vol = df["Close"], df["High"], df["Low"], df["Volume"]

    print("Computing indicators...")
    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()

    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()

    df["ATR"] = ta.volatility.AverageTrueRange(high, low, close, window=14).average_true_range()
    stoch = ta.momentum.StochasticOscillator(high, low, close)
    df["stoch_k"] = stoch.stoch()
    df["stoch_d"] = stoch.stoch_signal()
    df["CCI"] = ta.trend.CCIIndicator(high, low, close).cci()

    return df.dropna()

# --------------------
# Sequences and scaling
# --------------------
def make_sequences_and_scales(df, window=90, train_ratio=0.8):
    df_ind = compute_indicators(df)
    df_feat = df_ind[features].copy()
    n_features = len(features)

    X_raw, y_raw, last_close_raw, dates = [], [], [], []

    print("Creating sequences...")
    for i in tqdm(range(1, len(df_feat))):
        seq = df_feat.iloc[:i].values
        if len(seq) < window:
            pad_len = window - len(seq)
            seq = np.vstack([np.zeros((pad_len, n_features)), seq])
        else:
            seq = seq[-window:]
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])
        last_close_raw.append(df_feat["Close"].iloc[i - 1])
        dates.append(df_ind.index[i])

    X_raw = np.array(X_raw, dtype=np.float32)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)

    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = np.array(dates[split:], dtype="datetime64[ns]")

    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_raw.reshape(-1, n_features))
    X_train_scaled = feature_scaler.transform(X_train_raw.reshape(-1, n_features)).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_raw.reshape(-1, n_features)).reshape(X_test_raw.shape)

    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)
    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            feature_scaler, target_scaler, df_ind)

# --------------------
# Model
# --------------------
def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)

    x = Conv1D(128, 5, padding="causal", activation="relu")(inp)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)

    att = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)

    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)

    out = Dense(1)(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model

# --------------------
# Training & forecasting
# --------------------
def run(symbol="TATAPOWER", window=90, epochs=80, batch_size=32,
        force_download=False, n_forecast_days=30, ensemble_n=1):

    df = fetch_data(symbol, force_download=force_download)
    print("Data rows:", len(df))

    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     feature_scaler, target_scaler, df_ind) = make_sequences_and_scales(df, window)

    models = []
    for i in range(ensemble_n):
        model = build_model((X_train.shape[1], X_train.shape[2]))
        es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
        model.fit(X_train, y_train_scaled, validation_data=(X_test, y_test_scaled),
                  epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=2)
        models.append(model)

    preds = np.mean([m.predict(X_test).reshape(-1, 1) for m in models], axis=0)
    pred_price = target_scaler.inverse_transform(preds).flatten()
    y_test_price = y_test_raw

    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, last_close_test))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1]

    print(f"Model RMSE: {rmse_model:.4f}, Persistence RMSE: {rmse_persist:.4f}, Corr: {corr:.4f}")

    last_seq = df_ind[features].iloc[-window:].values
    last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)

    future_preds = []
    for _ in range(n_forecast_days):
        step_preds = np.mean([m.predict(last_seq_scaled) for m in models], axis=0)
        step_price = target_scaler.inverse_transform(step_preds).flatten()[0]
        future_preds.append(step_price)

        new_row = last_seq[-1].copy()
        if "Close" in features:
             new_row[features.index("Close")] = step_price
        last_seq = np.vstack([last_seq[1:], new_row])
        last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)

    future_dates = pd.date_range(df.index[-1], periods=n_forecast_days+1, freq="B")[1:]

    pattern_names = [attr for attr in dir(pta.cdl) if attr.startswith("cdl_")]
    pattern_signals = {}
    for pattern in pattern_names:
        func = getattr(pta.cdl, pattern)
        try:
            df_ind[pattern.upper()] = func(df_ind["Open"], df_ind["High"], df_ind["Low"], df_ind["Close"])
            pattern_signals[pattern.upper()] = df_ind[df_ind[pattern.upper()] != 0]
        except Exception as e:
            print(f"Could not compute pattern {pattern}: {e}")

    df_plot = df[df.index >= (df.index[-1] - pd.Timedelta(days=730))].copy()
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot.index,
        open=df_plot["Open"],
        high=df_plot["High"],
        low=df_plot["Low"],
        close=df_plot["Close"],
        name="Candlestick"
    )])

    for pattern_name, pattern_df in pattern_signals.items():
        bullish = pattern_df[pattern_df[pattern_name] > 0]
        bearish = pattern_df[pattern_df[pattern_name] < 0]

        fig.add_trace(go.Scatter(
            x=bullish.index,
            y=bullish["Close"],
            mode="markers",
            marker=dict(color="green", size=10, symbol="triangle-up"),
            name=f"Bullish {pattern_name}",
            text=[pattern_name]*len(bullish),
            hoverinfo="text+y"
        ))
        fig.add_trace(go.Scatter(
            x=bearish.index,
            y=bearish["Close"],
            mode="markers",
            marker=dict(color="red", size=10, symbol="triangle-down"),
            name=f"Bearish {pattern_name}",
            text=[pattern_name]*len(bearish),
            hoverinfo="text+y"
        ))

    fig.add_trace(go.Scatter(
        x=test_dates,
        y=pred_price,
        mode="lines",
        name="Predicted (Test)",
        line=dict(color="blue", dash="dot")
    ))
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=y_test_price,
        mode="lines",
        name="Actual (Test)",
        line=dict(color="orange")
    ))
    fig.add_trace(go.Scatter(
        x=future_dates,
        y=future_preds,
        mode="lines",
        name="Forecast (Next 30d)",
        line=dict(color="green", dash="dash")
    ))

    fig.update_layout(
        title=f"{symbol} Candlestick (Last 2 Years) with Predictions, 30-Day Forecast & All Patterns",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.show()

    return {"rmse_model": float(rmse_model), "rmse_persist": float(rmse_persist), "corr": float(corr)}

if __name__ == "__main__":
    stock_name = input("Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): ").strip() or "TATAPOWER"
    res = run(symbol=stock_name, window=90, epochs=80, batch_size=32,
              force_download=False, n_forecast_days=30, ensemble_n=1)
    print("Summary:", res)

Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): SUZLON
Loaded cached data: cache_data/SUZLON_NS_5y_1d.csv
Data rows: 1239
Computing indicators...
Creating sequences...


100%|██████████| 1205/1205 [00:00<00:00, 5787.34it/s]


Epoch 1/80
31/31 - 26s - 837ms/step - loss: 0.4338 - val_loss: 0.0715
Epoch 2/80
31/31 - 19s - 598ms/step - loss: 0.0300 - val_loss: 0.0051
Epoch 3/80
31/31 - 18s - 571ms/step - loss: 0.0183 - val_loss: 0.0030
Epoch 4/80
31/31 - 18s - 596ms/step - loss: 0.0193 - val_loss: 0.0038
Epoch 5/80
31/31 - 18s - 579ms/step - loss: 0.0127 - val_loss: 0.0064
Epoch 6/80
31/31 - 19s - 601ms/step - loss: 0.0093 - val_loss: 0.0324
Epoch 7/80
31/31 - 19s - 602ms/step - loss: 0.0061 - val_loss: 0.0060
Epoch 8/80
31/31 - 18s - 592ms/step - loss: 0.0062 - val_loss: 0.0224
Epoch 9/80
31/31 - 21s - 666ms/step - loss: 0.0067 - val_loss: 0.0194
Epoch 10/80
31/31 - 18s - 583ms/step - loss: 0.0065 - val_loss: 0.0051
Epoch 11/80
31/31 - 18s - 583ms/step - loss: 0.0052 - val_loss: 0.0236
Epoch 12/80
31/31 - 18s - 596ms/step - loss: 0.0053 - val_loss: 0.0295
Epoch 13/80
31/31 - 18s - 570ms/step - loss: 0.0047 - val_loss: 0.0185
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step
Model RMSE: 4.4813, Persistence RMSE: 1.5914, C

Summary: {'rmse_model': 4.481318159662413, 'rmse_persist': 1.591395565066379, 'corr': 0.6719611607602}


In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import pandas_ta as pta  # For candlestick pattern detection
import plotly.graph_objects as go
from tqdm import tqdm  # <-- Added for progress bars

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# --------------------
# Global Constants
# --------------------
CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200",
            "ATR", "stoch_k", "stoch_d", "CCI", "PE_Ratio", "PB_Ratio", "Market_Cap",
            "Dividend_Yield", "EPS"]
fundamental_features = ["PE_Ratio", "EPS", "PB_Ratio", "Market_Cap", "Dividend_Yield", "ROE", "ROCE"]
features += fundamental_features

# --------------------
# Fetch & cache data
# --------------------
def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol

    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")

    def download_and_cache():
        df_new = yf.download(yf_symbol, period=period, interval=interval, progress=False)
        if df_new is None or df_new.empty:
            raise ValueError(f"No data found for {yf_symbol}. Make sure the symbol is correct with '.NS' for NSE.")
        df_new.to_csv(cache_path)
        print(f"Downloaded and cached: {cache_path}")
        return df_new

    def validate_cache(df_cache):
        required_cols = ["Open", "High", "Low", "Close", "Volume"]
        if not all(col in df_cache.columns for col in required_cols):
            return False
        if df_cache[required_cols].isnull().all().any():
            return False
        return True

    if os.path.exists(cache_path) and not force_download:
        try:
            df = pd.read_csv(cache_path, index_col=0)
            df.index = pd.to_datetime(df.index, errors="coerce")
            df = df[~df.index.isna()]
            if not validate_cache(df):
                print(f"Cache corrupted or incomplete. Deleting and re-downloading: {cache_path}")
                os.remove(cache_path)
                df = download_and_cache()
            else:
                print(f"Loaded cached data: {cache_path}")
        except Exception:
            print(f"Failed to read cache. Deleting and re-downloading: {cache_path}")
            if os.path.exists(cache_path):
                os.remove(cache_path)
            df = download_and_cache()
    else:
        df = download_and_cache()

    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    df = df.dropna(subset=[col for col in required_cols if col in df.columns])

    if df.empty:
        raise ValueError(f"No valid data available for {yf_symbol} after cleaning.")
    return df

# --------------------
# Fetch Fundamental Data
# --------------------
def fetch_fundamental_data(symbol="TATAPOWER"):
    ticker = yf.Ticker(f"{symbol}.NS")
    info = ticker.info

    # Extract relevant fundamental metrics
    fundamentals = {
        "PE_Ratio": info.get("trailingPE", 0.0) or 0.0,
        "EPS": info.get("trailingEps", 0.0) or 0.0,
        "PB_Ratio": info.get("priceToBook", 0.0) or 0.0,
        "Market_Cap": info.get("marketCap", 0.0) or 0.0,
        "Dividend_Yield": info.get("dividendYield", 0.0) or 0.0,
        "ROE": info.get("returnOnEquity", 0.0) or 0.0,
        "ROCE": info.get("returnOnCapitalEmployed", 0.0) or 0.0
    }

    # Convert to DataFrame
    fundamental_df = pd.DataFrame(fundamentals, index=[0])
    return fundamental_df

# --------------------
# Compute indicators (technical + fundamental)
# --------------------
def compute_indicators(df, symbol="TATAPOWER"):
    df = df.copy()
    cols_to_coerce = ["Close", "High", "Low", "Volume", "Open"]
    for col in cols_to_coerce:
        if col in df.columns:
            val = df[col]
            if isinstance(val, pd.DataFrame):
                if col in val.columns:
                    val = val[col]
                else:
                    val = val.iloc[:, 0]
            df[col] = pd.to_numeric(val, errors="coerce")

    existing_cols = [c for c in cols_to_coerce if c in df.columns]
    if existing_cols:
        df = df.dropna(subset=existing_cols)

    close, high, low, vol = df["Close"], df["High"], df["Low"], df["Volume"]

    print("Computing indicators...")
    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()
    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()
    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()
    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()
    df["ATR"] = ta.volatility.AverageTrueRange(high, low, close, window=14).average_true_range()
    stoch = ta.momentum.StochasticOscillator(high, low, close)
    df["stoch_k"] = stoch.stoch()
    df["stoch_d"] = stoch.stoch_signal()
    df["CCI"] = ta.trend.CCIIndicator(high, low, close).cci()

    # Fetch and repeat fundamental data for all rows
    fundamental_df = fetch_fundamental_data(symbol)
    fundamental_df_repeated = pd.concat([fundamental_df] * len(df), ignore_index=True)
    df = pd.concat([df.reset_index(drop=True), fundamental_df_repeated], axis=1)

    return df.dropna()

# --------------------
# make_sequences_and_scales
# --------------------
def make_sequences_and_scales(df, window=90, train_ratio=0.8, symbol="TATAPOWER"):
    df_ind = compute_indicators(df, symbol=symbol)
    df_feat = df_ind[features].copy()
    n_features = len(features)

    X_raw, y_raw, last_close_raw, dates = [], [], [], []

    print("Creating sequences...")
    for i in tqdm(range(1, len(df_feat))):
        seq = df_feat.iloc[:i].values
        if len(seq) < window:
            pad_len = window - len(seq)
            seq = np.vstack([np.zeros((pad_len, n_features)), seq])
        else:
            seq = seq[-window:]
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])
        last_close_raw.append(df_feat["Close"].iloc[i - 1])
        dates.append(df_ind.index[i])

    X_raw = np.array(X_raw, dtype=np.float32)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)

    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = np.array(dates[split:], dtype="datetime64[ns]")

    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_raw.reshape(-1, n_features))
    X_train_scaled = feature_scaler.transform(X_train_raw.reshape(-1, n_features)).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_raw.reshape(-1, n_features)).reshape(X_test_raw.shape)

    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)
    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            feature_scaler, target_scaler, df_ind)

# --------------------
# Model
# --------------------
def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)

    x = Conv1D(128, 5, padding="causal", activation="relu")(inp)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)

    att = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)

    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)

    out = Dense(1)(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model


# --------------------
# Training & forecasting
# --------------------
def run(symbol="TATAPOWER", window=90, epochs=80, batch_size=32,
        force_download=False, n_forecast_days=30, ensemble_n=1):

    df = fetch_data(symbol, force_download=force_download)
    print("Data rows:", len(df))

    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     feature_scaler, target_scaler, df_ind) = make_sequences_and_scales(df, window)

    models = []
    for i in range(ensemble_n):
        model = build_model((X_train.shape[1], X_train.shape[2]))
        es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
        model.fit(X_train, y_train_scaled, validation_data=(X_test, y_test_scaled),
                  epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=2)
        models.append(model)

    # Test predictions
    preds = np.mean([m.predict(X_test).reshape(-1, 1) for m in models], axis=0)
    pred_price = target_scaler.inverse_transform(preds).flatten()
    y_test_price = y_test_raw

    # Evaluation metrics
    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, last_close_test))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1]
    print(f"Model RMSE: {rmse_model:.4f}, Persistence RMSE: {rmse_persist:.4f}, Corr: {corr:.4f}")

    # Forecast next 30 days
    last_seq = df_ind[features].iloc[-window:].values
    last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)
    future_preds = []

    for _ in range(n_forecast_days):
        step_preds = np.mean([m.predict(last_seq_scaled) for m in models], axis=0)
        step_price = target_scaler.inverse_transform(step_preds).flatten()[0]
        future_preds.append(step_price)

        new_row = last_seq[-1].copy()
        if "Close" in features:
             new_row[features.index("Close")] = step_price
        last_seq = np.vstack([last_seq[1:], new_row])
        last_seq_scaled = feature_scaler.transform(last_seq).reshape(1, window, -1)

    future_dates = pd.date_range(df.index[-1], periods=n_forecast_days+1, freq="B")[1:]

    # Candlestick chart for past 2 years + forecast
    df_plot = df[df.index >= (df.index[-1] - pd.Timedelta(days=730))].copy()
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot.index,
        open=df_plot["Open"],
        high=df_plot["High"],
        low=df_plot["Low"],
        close=df_plot["Close"],
        name="Candlestick"
    )])

    fig.add_trace(go.Scatter(
        x=test_dates,
        y=pred_price,
        mode="lines",
        name="Predicted (Test)",
        line=dict(color="blue", dash="dot")
    ))

    fig.add_trace(go.Scatter(
        x=future_dates,
        y=future_preds,
        mode="lines",
        name="Forecast (Next 30d)",
        line=dict(color="green", dash="dash")
    ))

    fig.update_layout(
        title=f"{symbol} Candlestick (Past 2 Years) & 30-Day Forecast",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.show()

    return {"rmse_model": float(rmse_model), "rmse_persist": float(rmse_persist), "corr": float(corr)}

if __name__ == "__main__":
    stock_name = input("Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): ").strip() or "TATAPOWER"
    res = run(symbol=stock_name, window=90, epochs=80, batch_size=32,
              force_download=False, n_forecast_days=30, ensemble_n=1)
    print("Summary:", res)

Enter stock symbol (e.g., TATAPOWER, RELIANCE, INFY): TATAPOWER
Loaded cached data: cache_data/TATAPOWER_NS_5y_1d.csv
Data rows: 1239
Computing indicators...
Creating sequences...


100%|██████████| 1205/1205 [00:00<00:00, 2269.97it/s]


Epoch 1/80
31/31 - 28s - 900ms/step - loss: 0.3861 - val_loss: 0.0043
Epoch 2/80
31/31 - 18s - 573ms/step - loss: 0.0357 - val_loss: 0.0110
Epoch 3/80
31/31 - 18s - 595ms/step - loss: 0.0205 - val_loss: 0.0053
Epoch 4/80
31/31 - 19s - 601ms/step - loss: 0.0157 - val_loss: 0.0077
Epoch 5/80
31/31 - 19s - 626ms/step - loss: 0.0146 - val_loss: 0.0074
Epoch 6/80
31/31 - 18s - 590ms/step - loss: 0.0121 - val_loss: 0.0495
Epoch 7/80
31/31 - 21s - 663ms/step - loss: 0.0137 - val_loss: 0.0463
Epoch 8/80
31/31 - 18s - 576ms/step - loss: 0.0101 - val_loss: 0.0118
Epoch 9/80
31/31 - 21s - 679ms/step - loss: 0.0091 - val_loss: 0.0443
Epoch 10/80
31/31 - 18s - 573ms/step - loss: 0.0094 - val_loss: 0.0699
Epoch 11/80
31/31 - 21s - 662ms/step - loss: 0.0084 - val_loss: 0.0369
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step
Model RMSE: 28.2162, Persistence RMSE: 6.4107, Corr: 0.5497
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━

Summary: {'rmse_model': 28.216236564421113, 'rmse_persist': 6.410748646739312, 'corr': 0.5496635222711693}


WITH UI TRIAL

In [ ]:
# ==============================================
# Install Gradio (only needed once in Colab)
# ==============================================
!pip install gradio --quiet

# ==============================================
# Your Existing Code (unchanged)
# ==============================================
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import pandas_ta as pta  # For candlestick pattern detection
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, Bidirectional, LSTM, Dropout,
                                     MultiHeadAttention, Add, LayerNormalization,
                                     GlobalAveragePooling1D, Dense)
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

CACHE_DIR = "cache_data"
REPORTS_DIR = "reports"
MODELS_DIR = "models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

features = ["Close", "RSI", "MACD", "MACD_signal", "BB_MID", "BB_UP", "BB_LOW",
            "ADX", "DI_POS", "DI_NEG", "OBV", "SMA_20", "EMA_20", "EMA_50", "EMA_200"]

def fetch_data(symbol="TATAPOWER", period="5y", interval="1d", force_download=False):
    if "." not in symbol:
        yf_symbol = f"{symbol}.NS"
    else:
        yf_symbol = symbol
    cache_path = os.path.join(CACHE_DIR, f"{yf_symbol.replace('.', '_')}_{period}_{interval}.csv")
    if os.path.exists(cache_path) and not force_download:
        df = pd.read_csv(cache_path, index_col=0)
        df.index = pd.to_datetime(df.index, errors="coerce")
        df = df[~df.index.isna()]
        print(f"Loaded cached data: {cache_path}")
    else:
        df = yf.download(yf_symbol, period=period, interval=interval, progress=False)
        if df is None or df.empty:
            raise ValueError(f"No data found for {yf_symbol}")
        df.to_csv(cache_path)
        print(f"Downloaded and cached: {cache_path}")
    required_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Missing column '{col}' in fetched data")
    df = df[required_cols].dropna()
    return df

def compute_indicators(df):
    df = df.copy()
    for col in ["Close", "High", "Low", "Volume", "Open"]:
        val = df[col]
        if isinstance(val, pd.DataFrame):
            if col in val.columns:
                val = val[col]
            else:
                val = val.iloc[:, 0]
        df[col] = pd.to_numeric(val, errors="coerce")
    df = df.dropna(subset=["Close", "High", "Low", "Volume"])
    close, high, low, vol = df["Close"], df["High"], df["Low"], df["Volume"]

    df["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
    df["BB_MID"] = bb.bollinger_mavg()
    df["BB_UP"] = bb.bollinger_hband()
    df["BB_LOW"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high, low, close, window=14)
    df["ADX"] = adx.adx()
    df["DI_POS"] = adx.adx_pos()
    df["DI_NEG"] = adx.adx_neg()

    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close, vol).on_balance_volume()
    df["SMA_20"] = close.rolling(20).mean()
    df["EMA_20"] = close.ewm(span=20, adjust=False).mean()
    df["EMA_50"] = close.ewm(span=50, adjust=False).mean()
    df["EMA_200"] = close.ewm(span=200, adjust=False).mean()

    return df.dropna()

def make_sequences_and_scales(df, window=90, train_ratio=0.8):
    df_ind = compute_indicators(df)
    df_feat = df_ind[features].copy()
    n_features = len(features)

    X_raw, y_raw, last_close_raw, dates = [], [], [], []
    for i in range(1, len(df_feat)):
        seq = df_feat.iloc[:i].values
        if len(seq) < window:
            pad_len = window - len(seq)
            seq = np.vstack([np.zeros((pad_len, n_features)), seq])
        else:
            seq = seq[-window:]
        X_raw.append(seq)
        y_raw.append(df_feat["Close"].iloc[i])
        last_close_raw.append(df_feat["Close"].iloc[i - 1])
        dates.append(df_ind.index[i])
    X_raw = np.array(X_raw, dtype=np.float32)
    y_raw = np.array(y_raw, dtype=np.float32).reshape(-1, 1)
    last_close_raw = np.array(last_close_raw, dtype=np.float32)

    split = int(len(X_raw) * train_ratio)
    X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
    y_train_raw, y_test_raw = y_raw[:split], y_raw[split:]
    last_close_test = last_close_raw[split:]
    test_dates = np.array(dates[split:], dtype="datetime64[ns]")

    feature_scaler = StandardScaler()
    feature_scaler.fit(X_train_raw.reshape(-1, n_features))
    X_train_scaled = feature_scaler.transform(X_train_raw.reshape(-1, n_features)).reshape(X_train_raw.shape)
    X_test_scaled = feature_scaler.transform(X_test_raw.reshape(-1, n_features)).reshape(X_test_raw.shape)

    target_scaler = MinMaxScaler()
    target_scaler.fit(y_train_raw)
    y_train_scaled = target_scaler.transform(y_train_raw).reshape(-1)
    y_test_scaled = target_scaler.transform(y_test_raw).reshape(-1)

    return (X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled,
            y_train_raw.flatten(), y_test_raw.flatten(), last_close_test, test_dates,
            feature_scaler, target_scaler, df_ind)

def build_model(input_shape, lr=1e-3):
    inp = Input(shape=input_shape)
    x = Conv1D(128, 5, padding="causal", activation="relu")(inp)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    att = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Add()([x, att])
    x = LayerNormalization()(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1)(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse")
    return model

def run(symbol="TATAPOWER", window=90, epochs=10, batch_size=32,
        force_download=False, n_forecast_days=30, ensemble_n=1):
    df = fetch_data(symbol, force_download=force_download)
    (X_train, y_train_scaled, X_test, y_test_scaled,
     y_train_raw, y_test_raw, last_close_test, test_dates,
     feature_scaler, target_scaler, df_ind) = make_sequences_and_scales(df, window)
    models = []
    for i in range(ensemble_n):
        model = build_model((X_train.shape[1], X_train.shape[2]))
        es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
        model.fit(X_train, y_train_scaled, validation_data=(X_test, y_test_scaled),
                  epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=0)
        models.append(model)
    preds = np.mean([m.predict(X_test).reshape(-1, 1) for m in models], axis=0)
    pred_price = target_scaler.inverse_transform(preds).flatten()
    y_test_price = y_test_raw
    rmse_model = np.sqrt(mean_squared_error(y_test_price, pred_price))
    rmse_persist = np.sqrt(mean_squared_error(y_test_price, last_close_test))
    corr = np.corrcoef(y_test_price, pred_price)[0, 1]
    return {"rmse_model": float(rmse_model), "rmse_persist": float(rmse_persist), "corr": float(corr)}

# ==============================================
# Gradio Frontend
# ==============================================
import gradio as gr

def gradio_runner(symbol, epochs, forecast_days):
    res = run(symbol=symbol, epochs=int(epochs), n_forecast_days=int(forecast_days))
    return f"✅ Model RMSE: {res['rmse_model']:.4f}\n" \
           f"📉 Persistence RMSE: {res['rmse_persist']:.4f}\n" \
           f"🔗 Correlation: {res['corr']:.4f}"

with gr.Blocks() as demo:
    gr.Markdown("## 📊 Stock Prediction with Candlestick Patterns")
    with gr.Row():
        stock = gr.Textbox(label="Stock Symbol (e.g. TATAPOWER, RELIANCE)", value="TATAPOWER")
        epochs = gr.Number(label="Epochs", value=10)
        forecast = gr.Number(label="Forecast Days", value=30)
    run_btn = gr.Button("Run Model")
    output = gr.Textbox(label="Results")
    run_btn.click(fn=gradio_runner, inputs=[stock, epochs, forecast], outputs=output)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5efa74814b99b100a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
